# Mini-Project
# Movie Recommendation System
### VIRAJ J RAUT(2131071)
### PRASAD Y SHELAR(2131077)

In [1]:
#All the header files required for the code
import numpy as np
import pandas as pd
!pip install factor_analyzer
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random

In [2]:
#Importing both the file using pandas 
data1 = pd.read_csv('movies data.csv')
data2 = pd.read_csv('ratings data.csv')

In [3]:
#Printing datasets
data1.head()

,Unnamed: 0,MovieID,Title,Genres
0,0,1,Toy Story (1995),Animation|Children's|Comedy
1,1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,2,3,Grumpier Old Men (1995),Comedy|Romance
3,3,4,Waiting to Exhale (1995),Comedy|Drama
4,4,5,Father of the Bride Part II (1995),Comedy


In [4]:
data2.head()

,Unnamed: 0,UserID,MovieID,Rating,Timestamp
0,0,1,1193,5,978300760
1,1,1,661,3,978302109
2,2,1,914,3,978301968
3,3,1,3408,4,978300275
4,4,1,2355,5,978824291


In [5]:
#Merging both the dataframes
data = pd.merge(data2 , data1 , how='outer', on='MovieID')

In [6]:
#getting first five record after merging datasets
data.head()

,Unnamed: 0_x,UserID,MovieID,Rating,Timestamp,Unnamed: 0_y,Title,Genres
0,0.0,1.0,1193,5.0,978300760.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama
1,120.0,2.0,1193,5.0,978298413.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama
2,1339.0,12.0,1193,4.0,978220179.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama
3,1518.0,15.0,1193,4.0,978199279.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama
4,1747.0,17.0,1193,5.0,978158471.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000386 entries, 0 to 1000385
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Unnamed: 0_x  1000209 non-null  float64
 1   UserID        1000209 non-null  float64
 2   MovieID       1000386 non-null  int64  
 3   Rating        1000209 non-null  float64
 4   Timestamp     1000209 non-null  float64
 5   Unnamed: 0_y  1000386 non-null  int64  
 6   Title         1000386 non-null  object 
 7   Genres        1000386 non-null  object 
dtypes: float64(4), int64(2), object(2)
memory usage: 68.7+ MB


In [8]:
data.describe()

,Unnamed: 0_x,UserID,MovieID,Rating,Timestamp,Unnamed: 0_y
count,1.000209e+06,1.000209e+06,1.000386e+06,1.000209e+06,1.000209e+06,1.000386e+06
mean,5.001040e+05,3.024512e+03,1.865526e+03,3.581564e+00,9.722437e+08,1.823025e+03
std,2.887356e+05,1.728413e+03,1.096030e+03,1.117102e+00,1.215256e+07,1.070164e+03
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08,0.000000e+00
25%,2.500520e+05,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08,1.017000e+03
50%,5.001040e+05,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08,1.767000e+03
75%,7.501560e+05,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08,2.701000e+03
max,1.000208e+06,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09,3.882000e+03


### Printing the data of Genres Attribute

In [9]:
data.Genres

0                   Drama
1                   Drama
2                   Drama
3                   Drama
4                   Drama
                ...      
1000381            Horror
1000382             Drama
1000383    Comedy|Romance
1000384             Drama
1000385             Drama
Name: Genres, Length: 1000386, dtype: object

In [10]:
# Converting Genres into different columns 
# Here we just create columns and put there initial value as 0
x = data.Genres
a = list()
for i in x:
    abc = i
    a.append(abc.split('|'))
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    data[i] = 0
data.head(2)

,Unnamed: 0_x,UserID,MovieID,Rating,Timestamp,Unnamed: 0_y,Title,Genres,Drama,Animation,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,0.0,1.0,1193,5.0,978300760.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,...,0,0,0,0,0,0,0,0,0,0
1,120.0,2.0,1193,5.0,978298413.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# we assign 1 to all the columns which are present in the Genres
for i in b:
    data.loc[data['Genres'].str.contains(i), i] = 1

In [12]:
data.head()

,Unnamed: 0_x,UserID,MovieID,Rating,Timestamp,Unnamed: 0_y,Title,Genres,Drama,Animation,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,0.0,1.0,1193,5.0,978300760.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,...,0,0,0,0,0,0,0,0,0,0
1,120.0,2.0,1193,5.0,978298413.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1339.0,12.0,1193,4.0,978220179.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1518.0,15.0,1193,4.0,978199279.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1747.0,17.0,1193,5.0,978158471.0,1176,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Now there is no use of genre 
# Since we have movie id so there is no need for movie names as well
data = data.drop(['Genres','Title'],axis =1)
data.head()

,Unnamed: 0_x,UserID,MovieID,Rating,Timestamp,Unnamed: 0_y,Drama,Animation,Musical,Action,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,0.0,1.0,1193,5.0,978300760.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,120.0,2.0,1193,5.0,978298413.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1339.0,12.0,1193,4.0,978220179.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1518.0,15.0,1193,4.0,978199279.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1747.0,17.0,1193,5.0,978158471.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#getting all attributes of dataset
data.columns

Index(['Unnamed: 0_x', 'UserID', 'MovieID', 'Rating', 'Timestamp',
       'Unnamed: 0_y', 'Drama', 'Animation', 'Musical', 'Action', 'Comedy',
       'Adventure', 'Romance', 'Children's', 'Thriller', 'Crime', 'Western',
       'Documentary', 'Mystery', 'Horror', 'Sci-Fi', 'Film-Noir', 'War',
       'Fantasy'],
      dtype='object')

In [15]:
# Because of merging some null values are created
data.isnull().sum()

Unnamed: 0_x    177
UserID          177
MovieID           0
Rating          177
Timestamp       177
Unnamed: 0_y      0
Drama             0
Animation         0
Musical           0
Action            0
Comedy            0
Adventure         0
Romance           0
Children's        0
Thriller          0
Crime             0
Western           0
Documentary       0
Mystery           0
Horror            0
Sci-Fi            0
Film-Noir         0
War               0
Fantasy           0
dtype: int64

In [16]:
#We simply drop the null values coz the are not treatable
data.dropna(inplace= True )

In [17]:
data.isnull().sum()

Unnamed: 0_x    0
UserID          0
MovieID         0
Rating          0
Timestamp       0
Unnamed: 0_y    0
Drama           0
Animation       0
Musical         0
Action          0
Comedy          0
Adventure       0
Romance         0
Children's      0
Thriller        0
Crime           0
Western         0
Documentary     0
Mystery         0
Horror          0
Sci-Fi          0
Film-Noir       0
War             0
Fantasy         0
dtype: int64

<h1>K-means Clustering</h1>

In [18]:
kmeanModel=KMeans(n_clusters=10)
kmeanModel.fit(data)

KMeans(n_clusters=10)

In [19]:
# Creating an extra column in data for storing the cluster values
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

413889    6
180963    6
841457    4
457505    6
998677    0
157717    4
661101    6
129619    6
841416    4
59909     4
Name: Cluster, dtype: int32

In [20]:
data.head()

,Unnamed: 0_x,UserID,MovieID,Rating,Timestamp,Unnamed: 0_y,Drama,Animation,Musical,Action,...,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy,Cluster
0,0.0,1.0,1193,5.0,978300760.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6
1,120.0,2.0,1193,5.0,978298413.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2,1339.0,12.0,1193,4.0,978220179.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6
3,1518.0,15.0,1193,4.0,978199279.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,1747.0,17.0,1193,5.0,978158471.0,1176,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [21]:
#allocation of a single cluster to a movie 
e = []
def fi(group):
    a = pd.DataFrame(group)
    b = pd.DataFrame(a['Cluster'].value_counts())
    d = a.index 
    c = [a['MovieID'][d[0]],int(b.idxmax())]
    e.append(c)
    

In [22]:
#applying group by fuction on movieID attributes.
data.groupby("MovieID").apply(lambda x: fi(x))

""


In [23]:
e = pd.DataFrame(e)

In [24]:
# Here just the column names are swapped
e.rename(columns = {0:'MovieID',1:'Cluster'},inplace=True)
e.drop_duplicates(inplace=True)
e.head(10)

,MovieID,Cluster
0,1,6
1,2,6
2,3,6
3,4,6
4,5,6
5,6,6
6,7,6
7,8,6
8,9,6
9,10,6


In [25]:
data1 = pd.read_csv('movies data.csv')
new_data = pd.merge(e , data1 , how='outer', on='MovieID')

In [26]:
#This function select the cluster for a user according the the user choice
def select_c():
    global l
    print('Select The Movies Id you would like to watch:')
    l=[]
    for i in range(15):
        l.append(random.randint(0,3883))
    for i in l:
        print(new_data['MovieID'][i] , new_data['Title'][i],sep='--->')
    print('--------------------------------------------------------------------')
    l = int(input())
    l = new_data['Cluster'][new_data.MovieID == l]

In [27]:
# This is the main function which recommend you movies
def main():
    ans = False
    while not ans:
        select_c()
        print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
        print('--------------------------------------------------------------------')
        print('Do you like these movies(y/n)')
        abc = input()
        while ((abc =='y') or (abc == 'Y')):          
            print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
            print('--------------------------------------------------------------------')
            print('Want more!!!!(y/n)')
            abc = input()
            if ((abc =='N') or (abc == 'n')):
                ans =True

In [28]:
main()

Select The Movies Id you would like to watch:
2836--->Outside Providence (1999)
3862--->About Adam (2000)
3028--->Taming of the Shrew, The (1967)
1337--->Body Snatcher, The (1945)
1467--->Salut cousin! (1996)
3018--->Re-Animator (1985)
2537--->Beyond the Poseidon Adventure (1979)
1254--->Treasure of the Sierra Madre, The (1948)
3724--->Coming Home (1978)
288--->Natural Born Killers (1994)
3902--->Goya in Bordeaux (Goya en Bodeos) (1999)
3287--->Tigger Movie, The (2000)
2389--->Psycho (1998)
418--->Being Human (1993)
2946--->Help! (1965)
--------------------------------------------------------------------
2946
63                                        Bio-Dome (1996)
689     Wallace & Gromit: The Best of Aardman Animatio...
3336                                   Carnosaur 2 (1995)
2053                                Broadcast News (1987)
2159                                Big Chill, The (1983)
3154                                    Volunteers (1985)
2630                            Out

Analysis: K-means clustering is better approach for movie recommendation, it helps to find similar movie with the help of MovieID. Clustering is used to grouping the data together and find the similarity beteen them i.e Recommendation system.
    